# Import Libraries

In [2]:
import pandas as pd
from pandas_profiling import ProfileReport
import requests
from fake_useragent import UserAgent
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import bs4
import time

# Read Dataset

In [55]:
df = pd.read_excel('data.xlsx')
df.head()

,status,card_present_flag,bpay_biller_code,account,currency,long_lat,txn_description,merchant_id,merchant_code,first_name,...,age,merchant_suburb,merchant_state,extraction,amount,transaction_id,country,customer_id,merchant_long_lat,movement
0,authorized,1.0,NaN,ACC-1598451071,AUD,153.41 -27.95,POS,81c48296-73be-44a7-befa-d053f48ce7cd,NaN,Diana,...,26,Ashmore,QLD,2018-08-01T01:01:15.000+0000,16.25,a623070bfead4541a6b0fff8a09e706c,Australia,CUS-2487424745,153.38 -27.99,debit
1,authorized,0.0,NaN,ACC-1598451071,AUD,153.41 -27.95,SALES-POS,830a451c-316e-4a6a-bf25-e37caedca49e,NaN,Diana,...,26,Sydney,NSW,2018-08-01T01:13:45.000+0000,14.19,13270a2a902145da9db4c951e04b51b9,Australia,CUS-2487424745,151.21 -33.87,debit
2,authorized,1.0,NaN,ACC-1222300524,AUD,151.23 -33.94,POS,835c231d-8cdf-4e96-859d-e9d571760cf0,NaN,Michael,...,38,Sydney,NSW,2018-08-01T01:26:15.000+0000,6.42,feb79e7ecd7048a5a36ec889d1a94270,Australia,CUS-2142601169,151.21 -33.87,debit
3,authorized,1.0,NaN,ACC-1037050564,AUD,153.10 -27.66,SALES-POS,48514682-c78a-4a88-b0da-2d6302e64673,NaN,Rhonda,...,40,Buderim,QLD,2018-08-01T01:38:45.000+0000,40.90,2698170da3704fd981b15e64a006079e,Australia,CUS-1614226872,153.05 -26.68,debit
4,authorized,1.0,NaN,ACC-1598451071,AUD,153.41 -27.95,SALES-POS,b4e02c10-0852-4273-b8fd-7b3395e32eb0,NaN,Diana,...,26,Mermaid Beach,QLD,2018-08-01T01:51:15.000+0000,3.25,329adf79878c4cf0aeb4188b4691c266,Australia,CUS-2487424745,153.44 -28.06,debit


# Feature Engineering (State from Latitude & Longitude of customer)

In [56]:
# We can get state for some of long_lat using (merchant_long_lat, merchant_state) combo.
# Out of 100 long_lat, I was able to retrieve 32 using this method.
# rest 68 long_lat need to be scraped.
k=list(set(df['merchant_long_lat']) & set(df['long_lat']))
l=df.loc[df['merchant_long_lat'].isin(k)][['merchant_long_lat','merchant_state']].drop_duplicates()
d1=dict(zip(l['merchant_long_lat'],l['merchant_state']))
rem=set(df['long_lat'].replace(d1).unique())-set(d1.values())
len(d1),len(rem)

(32, 68)

## Web Scraping

In [9]:
driver = webdriver.Chrome(ChromeDriverManager().install())
url1="https://www.latlong.net/user/login"
url2="https://www.latlong.net/Show-Latitude-Longitude.html"

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280


[WDM] - Driver [C:\Users\Amit\.wdm\drivers\chromedriver\win32\87.0.4280.88\chromedriver.exe] found in cache


In [10]:
# Log into website
driver.get(url1)
username = driver.find_element_by_id("email")
username.clear()
username.send_keys("xecixi7482@maksap.com")

password = driver.find_element_by_id("password1")
password.clear()
password.send_keys("apar123")

driver.find_element_by_tag_name("button").click()

In [ ]:
# search and store State using Latitude and Longitude values
driver.get(url2)
d2={}
for i in rem:
    try:
        x = driver.find_element_by_id("latitude")
        x.clear()
        x.send_keys(i.split()[1])
        
        time.sleep(5)
        
        y = driver.find_element_by_id("longitude")
        y.clear()
        y.send_keys(i.split()[0])
        
        time.sleep(5)
        
        driver.find_element_by_tag_name("button").click()
        
        time.sleep(5)
        
        source = driver.page_source
        soup=bs4.BeautifulSoup(source, 'html.parser')
        txt=soup.find('div',{'class':'bgw shadow padding10','id':'address'}).text
        d2[i]=txt.split(',')[3].strip()
        print(i,txt.split(',')[3].strip())
        
    except:
        pass

In [49]:
d1.update(d2)
d1

{'150.82 -34.01': '2566Varroville Sydney  Australia',
 '115.90 -31.92': 'Western Australia',
 '146.94 -36.04': 'New South Wales',
 '150.90 -33.78': 'New South Wales',
 '115.79 -31.94': 'New South Wales',
 '144.66 -37.91': 'New South Wales',
 '151.23 -33.94': 'New South Wales',
 '150.50 -23.40': 'New South Wales',
 '144.99 -37.90': 'New South Wales',
 '151.27 -33.85': 'New South Wales',
 '147.31 -42.88': 'New South Wales',
 '144.79 -37.73': 'New South Wales',
 '130.90 -12.37': 'New South Wales',
 '114.62 -28.80': 'New South Wales',
 '151.04 -33.80': 'New South Wales',
 '147.63 -22.84': 'New South Wales',
 '151.04 -33.77': 'New South Wales',
 '151.09 -33.91': 'New South Wales',
 '115.79 -31.79': 'New South Wales',
 '153.41 -28.01': 'QLD',
 '115.81 -31.82': 'WA',
 '151.27 -33.76': 'NSW',
 '151.22 -33.87': 'NSW',
 '115.74 -31.72': 'WA',
 '153.03 -27.51': 'QLD',
 '130.84 -12.45': 'NT',
 '145.03 -37.91': 'VIC',
 '145.16 -37.84': 'VIC',
 '145.17 -37.84': 'VIC',
 '144.96 -37.84': 'VIC',
 '144.

In [50]:
df['customer_state']=df['long_lat'].replace(d1)
df['customer_state']=df['customer_state'].replace({'New South Wales':'NSW',
                                      'Victoria':'VIC',
                                      'Queensland':'QLD',
                                      'South Australia':'SA',
                                      'Western Australia':'WA',
                                      'Northern Territory':'NT',
                                      '2566Varroville Sydney  Australia':'NSW',
                                      'Australian Capital Territory':'ACT'})

In [51]:
df['customer_state'].value_counts()

NSW    5763
VIC    2366
QLD    1900
WA     1185
SA      595
NT      194
ACT      40
Name: customer_state, dtype: int64

# Save Output

In [54]:
df.drop('long_lat',axis=1).to_excel('new_data.xlsx',index=False)